In [35]:
# check what directory I am in
# should be "code" directory in my blog draft post
import os
from dotenv import load_dotenv

print(os.getcwd())
load_dotenv('.env', override=True)


/home/brian/Projects/learning-with-code/content/drafts/read-database-python-odbc/code


True

In [36]:
userid = os.environ.get('DB_UID')
password = os.environ.get('DB_PWD')
db_server_name = os.environ.get('DB_SERVER')
db_name = os.environ.get('DB_NAME')

In [37]:
print(userid)
print(password)
print(db_server_name)
print(db_name)


CloudSA18f60e2c
3ANq!vyhb9qA
brian-dbserver.database.windows.net
data-science-test


In [38]:
connection_string = (
    "Driver={ODBC Driver 18 for SQL Server};"+
    "Server=tcp:"+db_server_name+",1433;"+
    "Database="+db_name+";"+
    "Uid="+userid+";"+
    "Pwd="+password+";"+
    "Encrypt=yes;"+
    "TrustServerCertificate=no;"+
    "Connection Timeout=30;"
)

In [39]:
print(connection_string)

Driver={ODBC Driver 18 for SQL Server};Server=tcp:brian-dbserver.database.windows.net,1433;Database=data-science-test;Uid=CloudSA18f60e2c;Pwd=3ANq!vyhb9qA;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;


In [40]:
import pyodbc
conn = pyodbc.connect(connection_string)
print(conn)

In [41]:
statement = """
SELECT DISTINCT
  TABLE_SCHEMA
FROM INFORMATION_SCHEMA.VIEWS
ORDER BY TABLE_SCHEMA
"""
cursor = conn.cursor()
cursor.execute(statement)
print(cursor.fetchall())
cursor.close()

[('SalesLT',), ('sys',)]


In [42]:
statement = """
SELECT DISTINCT
  TABLE_SCHEMA
FROM INFORMATION_SCHEMA.TABLES
ORDER BY TABLE_SCHEMA
"""
cursor = conn.cursor()
cursor.execute(statement)
print(cursor.fetchall())
cursor.close()

[('dbo',), ('SalesLT',), ('sys',)]


In [43]:
statement = """
SELECT
  TABLE_NAME
FROM INFORMATION_SCHEMA.VIEWS
WHERE TABLE_SCHEMA = 'SalesLT'
ORDER BY TABLE_NAME
"""
cursor = conn.cursor()
cursor.execute(statement)
print(cursor.fetchall())
cursor.close()

[('vGetAllCategories',), ('vProductAndDescription',), ('vProductModelCatalogDescription',)]


In [44]:
statement = """
SELECT
  TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'SalesLT'
ORDER BY TABLE_NAME
"""
cursor = conn.cursor()
cursor.execute(statement)
print(cursor.fetchall())
cursor.close()

[('Address',), ('Customer',), ('CustomerAddress',), ('Product',), ('ProductCategory',), ('ProductDescription',), ('ProductModel',), ('ProductModelProductDescription',), ('SalesOrderDetail',), ('SalesOrderHeader',), ('vGetAllCategories',), ('vProductAndDescription',), ('vProductModelCatalogDescription',)]


In [45]:
cursor = conn.cursor()
for table_name in cursor.tables():
    print(table_name)

('data-science-test', 'dbo', 'BuildVersion', 'TABLE', None)
('data-science-test', 'dbo', 'ErrorLog', 'TABLE', None)
('data-science-test', 'SalesLT', 'Address', 'TABLE', None)
('data-science-test', 'SalesLT', 'Customer', 'TABLE', None)
('data-science-test', 'SalesLT', 'CustomerAddress', 'TABLE', None)
('data-science-test', 'SalesLT', 'Product', 'TABLE', None)
('data-science-test', 'SalesLT', 'ProductCategory', 'TABLE', None)
('data-science-test', 'SalesLT', 'ProductDescription', 'TABLE', None)
('data-science-test', 'SalesLT', 'ProductModel', 'TABLE', None)
('data-science-test', 'SalesLT', 'ProductModelProductDescription', 'TABLE', None)
('data-science-test', 'SalesLT', 'SalesOrderDetail', 'TABLE', None)
('data-science-test', 'SalesLT', 'SalesOrderHeader', 'TABLE', None)
('data-science-test', 'sys', 'trace_xe_action_map', 'TABLE', None)
('data-science-test', 'sys', 'trace_xe_event_map', 'TABLE', None)
('data-science-test', 'INFORMATION_SCHEMA', 'CHECK_CONSTRAINTS', 'VIEW', None)
('data-s

In [46]:
cursor = conn.cursor()

for row in cursor.tables(schema='SalesLT'):
    print(*row, sep=', ')

# Can only get columns after I get data in the cursor
columns = [column[0] for column in cursor.description]
print(*columns, sep=', ')

cursor.close()

data-science-test, SalesLT, Address, TABLE, None
data-science-test, SalesLT, Customer, TABLE, None
data-science-test, SalesLT, CustomerAddress, TABLE, None
data-science-test, SalesLT, Product, TABLE, None
data-science-test, SalesLT, ProductCategory, TABLE, None
data-science-test, SalesLT, ProductDescription, TABLE, None
data-science-test, SalesLT, ProductModel, TABLE, None
data-science-test, SalesLT, ProductModelProductDescription, TABLE, None
data-science-test, SalesLT, SalesOrderDetail, TABLE, None
data-science-test, SalesLT, SalesOrderHeader, TABLE, None
data-science-test, SalesLT, vGetAllCategories, VIEW, None
data-science-test, SalesLT, vProductAndDescription, VIEW, None
data-science-test, SalesLT, vProductModelCatalogDescription, VIEW, None
table_cat, table_schem, table_name, table_type, remarks


In [47]:
cursor = conn.cursor()

column_list = (
    cursor.columns(
        table='Address', 
        schema='SalesLT')
    .fetchall()
)

# headers
headers = [h[0] for h in cursor.description]
print(f'{headers[3]:{25}}{headers[5]:{20}}{headers[6]:{11}}')

# data
for row in column_list:
    print(f'{row[3]:{25}}{row[5]:{20}}{row[6]:{6}}')

cursor.close()

column_name              type_name           column_size
AddressID                int identity            10
AddressLine1             nvarchar                60
AddressLine2             nvarchar                60
City                     nvarchar                30
StateProvince            Name                    50
CountryRegion            Name                    50
PostalCode               nvarchar                15
rowguid                  uniqueidentifier        36
ModifiedDate             datetime                23


In [48]:
cursor = conn.cursor()

table = 'Address'
schema = 'SalesLT'

primary_keys = cursor.primaryKeys(table=table, schema=schema).fetchall()
print(f"{table}:  Primary Keys = {primary_keys}")

foreign_keys = cursor.foreignKeys(table=table, schema=schema).fetchall()
print(f"{table}:  Foreign Keys = {foreign_keys}")

cursor.close()

Address:  Primary Keys = [('data-science-test', 'SalesLT', 'Address', 'AddressID', 1, 'PK_Address_AddressID')]
Address:  Foreign Keys = [('data-science-test', 'SalesLT', 'Address', 'AddressID', 'data-science-test', 'SalesLT', 'CustomerAddress', 'AddressID', 1, 1, 1, 'FK_CustomerAddress_Address_AddressID', 'PK_Address_AddressID', 7), ('data-science-test', 'SalesLT', 'Address', 'AddressID', 'data-science-test', 'SalesLT', 'SalesOrderHeader', 'BillToAddressID', 1, 1, 1, 'FK_SalesOrderHeader_Address_BillTo_AddressID', 'PK_Address_AddressID', 7), ('data-science-test', 'SalesLT', 'Address', 'AddressID', 'data-science-test', 'SalesLT', 'SalesOrderHeader', 'ShipToAddressID', 1, 1, 1, 'FK_SalesOrderHeader_Address_ShipTo_AddressID', 'PK_Address_AddressID', 7)]


In [49]:
cursor = conn.cursor()

column_list = (
    cursor.columns(
        table='vProductAndDescription', 
        schema='SalesLT')
    .fetchall()
)

# headers
headers = [h[0] for h in cursor.description]
print(f'{headers[3]:{25}}{headers[5]:{20}}{headers[6]:{11}}')

# data
for row in column_list:
    print(f'{row[3]:{25}}{row[5]:{20}}{row[6]:{6}}')

cursor.close()

column_name              type_name           column_size
ProductID                int                     10
Name                     Name                    50
ProductModel             Name                    50
Culture                  nchar                    6
Description              nvarchar               400
